In [ ]:
import psycopg2

conn = psycopg2.connect(
    dbname="rag_chatbot",
    user="postgres",
    password="MY_PASSWORD",
    host="localhost",
    port="5432"
)

conn.autocommit = True  # <-- Important
cur = conn.cursor()
print("✅ Connected with autocommit")


✅ Connected with autocommit


In [4]:
# ==========================
# Imports
# ==========================
import psycopg2
from psycopg2.extras import execute_values
from pathlib import Path
import os
from dotenv import load_dotenv
import PyPDF2  # pour lire les PDF
from langchain_mistralai import MistralAIEmbeddings

# ==========================
# Variables
# ==========================
conversation_file_path = "../data/Pres_Accueil_UBS.pdf"  # chemin vers ton PDF
env_path = Path(os.getcwd()).parent / "src" / ".env"

if env_path.exists():
    load_dotenv(dotenv_path=env_path)

MISTRAL_API_KEY = os.environ.get("MISTRAL_API_KEY")
db_connection_str = "dbname=rag_chatbot user=postgres password=123456yz host=localhost port=5432"


In [5]:
# ==========================
# Fonction pour lire le PDF
# ==========================
def extract_text_from_pdf(pdf_path: str) -> list[str]:
    corpus_list = []
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text = page.extract_text()
            if text:
                # split par lignes et nettoyer
                lines = [line.strip() for line in text.split("\n") if line.strip()]
                corpus_list.extend(lines)
    return corpus_list

corpus_list = extract_text_from_pdf(conversation_file_path)
print(f"✅ {len(corpus_list)} lignes extraites du PDF")
print(corpus_list[:5])


✅ 346 lignes extraites du PDF
['Corpus Accueil_UBS', 'Présentation générale', 'Jean-Yves Antoine 1, Judith Muzerelle 2', '1LI – Université François Rabelais de Tours', '2LLL – Université d’Orléans']


In [6]:
# ==========================
# Connexion à la DB
# ==========================
conn = psycopg2.connect(db_connection_str)
cur = conn.cursor()
conn.autocommit = True

# Créer l'extension vector si elle n'existe pas
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")

# Supprimer la table existante si besoin
cur.execute("DROP TABLE IF EXISTS embeddings")

# Créer la table embeddings avec vecteur de dimension 768 (MistralAI par défaut)
cur.execute("""
CREATE TABLE embeddings (
    id SERIAL PRIMARY KEY,
    corpus TEXT,
    embedding VECTOR(768)
)
""")
print("✅ Table embeddings créée avec VECTOR(768)")


✅ Table embeddings créée avec VECTOR(768)


In [7]:
# ==========================
# Fonction pour lire le PDF
# ==========================
def extract_text_from_pdf(pdf_path: str) -> list[str]:
    corpus_list = []
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text = page.extract_text()
            if text:
                # split par lignes et nettoyer
                lines = [line.strip() for line in text.split("\n") if line.strip()]
                corpus_list.extend(lines)
    return corpus_list

corpus_list = extract_text_from_pdf(conversation_file_path)
print(f"✅ {len(corpus_list)} lignes extraites du PDF")
print(corpus_list[:5])


✅ 346 lignes extraites du PDF
['Corpus Accueil_UBS', 'Présentation générale', 'Jean-Yves Antoine 1, Judith Muzerelle 2', '1LI – Université François Rabelais de Tours', '2LLL – Université d’Orléans']


In [8]:
# ==========================
# Initialiser MistralAI Embeddings
# ==========================
mistral_client = MistralAIEmbeddings(api_key=MISTRAL_API_KEY)
print("✅ Client MistralAI prêt")


d:\Users\zouar\OneDrive\Documents\ProjetRag\Chatbot-RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Client MistralAI prêt


In [9]:
# ==========================
# Fonction pour calculer et insérer un embedding
# ==========================
def save_embedding(corpus: str, client: MistralAIEmbeddings, cursor):
    embedding = client.embed_query(corpus)  # liste de floats
    cursor.execute(
        "INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)",
        (corpus, embedding)
    )


In [11]:
# Supprimer la table existante si besoin
cur.execute("DROP TABLE IF EXISTS embeddings")

# Créer la table embeddings avec vecteur de dimension 1024 (Mistral)
cur.execute("""
CREATE TABLE embeddings (
    id SERIAL PRIMARY KEY,
    corpus TEXT,
    embedding VECTOR(1024)
)
""")

print("✅ Table embeddings créée avec VECTOR(1024)")


✅ Table embeddings créée avec VECTOR(1024)


In [12]:
import time

# ==========================
# Boucle d'insertion
# ==========================
batch_size = 1       # 1 ligne à la fois pour éviter quota
pause_seconds = 1    # pause 1 sec entre chaque requête

for i, corpus in enumerate(corpus_list):
    save_embedding(corpus, mistral_client, cur)
    if (i+1) % 10 == 0:
        print(f"{i+1} embeddings insérés...")
    time.sleep(pause_seconds)

conn.commit()
print("✅ Tous les embeddings insérés")


10 embeddings insérés...
20 embeddings insérés...
30 embeddings insérés...
40 embeddings insérés...
50 embeddings insérés...
60 embeddings insérés...
70 embeddings insérés...
80 embeddings insérés...
90 embeddings insérés...
100 embeddings insérés...
110 embeddings insérés...
120 embeddings insérés...
130 embeddings insérés...
140 embeddings insérés...
150 embeddings insérés...
160 embeddings insérés...
170 embeddings insérés...
180 embeddings insérés...
190 embeddings insérés...
200 embeddings insérés...
210 embeddings insérés...
220 embeddings insérés...
230 embeddings insérés...
240 embeddings insérés...
250 embeddings insérés...
260 embeddings insérés...
270 embeddings insérés...
280 embeddings insérés...
290 embeddings insérés...
300 embeddings insérés...
310 embeddings insérés...
320 embeddings insérés...
330 embeddings insérés...
340 embeddings insérés...
✅ Tous les embeddings insérés


In [29]:
cur.execute("SELECT corpus, embedding FROM embeddings LIMIT 1;")
row = cur.fetchone()

print("Texte :", row[0])
print("Embedding (vecteur) :")
print(row[1])
print("Longueur du vecteur :", len(row[1]))


Texte : Corpus Accueil_UBS
Embedding (vecteur) :
[-0.024276733,-0.009941101,0.033599854,-0.033416748,0.004924774,0.00774765,-0.013725281,0.0040359497,-0.00793457,0.019515991,-0.026504517,0.055633545,0.025024414,-0.0010499954,0.00774765,-0.021377563,0.03137207,-0.0015516281,-0.04107666,-0.01838684,0.020904541,0.0129776,-0.027816772,-0.029678345,-0.04928589,0.017822266,0.008911133,-0.020263672,-0.015686035,-0.024459839,0.01260376,0.00093364716,0.0057640076,0.027069092,-0.01763916,0.001405716,0.032684326,0.0075149536,-0.019607544,-0.0011491776,0.003780365,-0.047607422,-0.04928589,-0.055267334,0.030426025,-0.02482605,0.031173706,0.010498047,-0.04611206,0.003873825,0.013160706,0.06048584,-0.03173828,-0.045166016,-0.033599854,0.028381348,0.0065345764,0.00630188,-0.05114746,0.07171631,-0.0496521,0.0067214966,-0.05078125,-0.028747559,0.0446167,0.016143799,0.026138306,-0.03286743,-0.006626129,-0.008071899,-0.0022640228,0.011390686,-0.0073280334,-0.008491516,0.010551453,-0.03378296,-0.041992188,

In [23]:
import numpy as np
from psycopg2.extensions import AsIs, register_adapter

# Adapter to let psycopg2 handle numpy arrays
def adapt_numpy_array(numpy_array):
    return AsIs("ARRAY[%s]" % ",".join(map(str, numpy_array)))
register_adapter(np.ndarray, adapt_numpy_array)

def search_similar_docs(query, embedding_model, cursor, top_k=3):
    # 1) Convert query to embedding vector (as numpy array)
    query_embedding = np.array(embedding_model.embed_query(query))
    
    # 2) Search vector in PostgreSQL with explicit cast to vector
    cursor.execute(
        """
        SELECT corpus, embedding, embedding <-> %s::vector AS distance
        FROM embeddings
        ORDER BY embedding <-> %s::vector
        LIMIT %s;
        """,
        (query_embedding, query_embedding, top_k)
    )
    
    return cursor.fetchall()


In [44]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Chemin vers ton .env
env_path = Path(os.getcwd()).parent / "src" / ".env"
if env_path.exists():
    load_dotenv(dotenv_path=env_path)

# Charger la clé API pour le LLM
LLM_API_KEY = os.environ.get("MISTRAL_API_KEY")
if not LLM_API_KEY:
    raise ValueError("❌ LLM_API_KEY non trouvé dans .env")
print("✅ Clé LLM chargée :", LLM_API_KEY[:10] + "...")


✅ Clé LLM chargée : QO8zDcbg3Y...


In [45]:
from langchain_mistralai import MistralAIEmbeddings, ChatMistralAI

# Initialiser l'embedding model
embedding_model = MistralAIEmbeddings(api_key=LLM_API_KEY)

# Initialiser le chat model
chat_model = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    api_key=LLM_API_KEY
)

In [46]:
def rag_chat(query, top_k=3):
    # 1) Récupérer les documents similaires
    results = search_similar_docs(query, embedding_model, cur, top_k=top_k)
    
    # 2) Construire le contexte
    context = "\n".join([row[0] for row in results])
    
    # 3) Préparer le prompt
    prompt = f"Voici des informations extraites du PDF :\n{context}\n\nQuestion : {query}\nRéponse :"
    
    # 4) Générer la réponse avec invoke() au lieu de predict()
    response = chat_model.invoke(prompt)
    
    # 5) Extraire le contenu
    answer = response.content
    
    return answer

print("✅ Fonction RAG définie")

✅ Fonction RAG définie


In [47]:
while True:
    user_query = input("\n Quel est l’objectif du corpus Accueil_UBS ")
    if user_query.lower() == "exit":
        break
    
    answer = rag_chat(user_query)
    print("\n🤖 Chatbot :", answer)



🤖 Chatbot : Voici une réponse structurée à partir des informations extraites du PDF concernant le **projet AGILE-OURAL** et la transcription dans ce cadre :

---

### **Réponse détaillée :**

#### **1. Contexte du projet AGILE-OURAL**
Le projet **AGILE-OURAL** s’inscrit dans le **programme TECHNOLANGUE**, financé par le **Ministère de la Recherche** français. Ce programme visait à développer des technologies linguistiques avancées, notamment pour le traitement automatique des langues (TAL) et la transcription de données orales.

#### **2. Objectifs de la transcription**
Les transcriptions réalisées dans le cadre d’AGILE-OURAL se distinguent par :
- **Des conventions particulières** : Elles sont conçues pour **capturer avec précision** les nuances de la parole (hésitations, intonations, chevauchements, etc.), souvent absentes des transcriptions standard.
- **Un format adapté** : L’objectif est de fournir des données exploitables pour des applications en **linguistique computationnelle*